In [2]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data
warnings.filterwarnings('ignore')
#pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
ticker = "^SPX"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [4]:
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-09-30 00:00:00-04:00,5726.520020,5765.140137,5703.529785,5762.479980,4425730000,0.0,0.0
2024-10-01 00:00:00-04:00,5757.729980,5757.729980,5681.279785,5708.750000,4025180000,0.0,0.0
2024-10-02 00:00:00-04:00,5698.140137,5719.629883,5674.000000,5709.540039,3829050000,0.0,0.0


In [13]:
strategy = SmaCross
strategy.n1 =42
strategy.n2 = 242
bt_sma = Backtest(data["2019":], SmaCross, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_sma.run()
stats

Start                     2019-01-02 00:00...
End                       2024-10-04 00:00...
Duration                   2101 days 23:00:00
Exposure Time [%]                   78.551724
Equity Final [$]                 15247.656591
Equity Peak [$]                  15372.736669
Return [%]                          52.476566
Buy & Hold Return [%]              129.123547
Return (Ann.) [%]                     7.60673
Volatility (Ann.) [%]               12.909409
Sharpe Ratio                         0.589239
Sortino Ratio                        0.923041
Calmar Ratio                         0.347682
Max. Drawdown [%]                  -21.878405
Avg. Drawdown [%]                   -1.789704
Max. Drawdown Duration      295 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                      42.595624
Worst Trade [%]                    -14.868977
Avg. Trade [%]                    

In [14]:
bt_sma.plot()

Row(id='3646', ...)

In [18]:
%%time
stats = bt_sma.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 2.97 s, sys: 54.2 ms, total: 3.02 s
Wall time: 3 s


Start                     2020-07-10 00:00...
End                       2024-08-06 00:00...
Duration                   1488 days 00:00:00
Exposure Time [%]                       95.71
Equity Final [$]                    12,462.11
Equity Peak [$]                     12,537.98
Return [%]                              24.62
Buy & Hold Return [%]                   64.52
Return (Ann.) [%]                        5.56
Volatility (Ann.) [%]                   14.04
Sharpe Ratio                             0.40
Sortino Ratio                            0.61
Calmar Ratio                             0.36
Max. Drawdown [%]                      -15.27
Avg. Drawdown [%]                       -2.37
Max. Drawdown Duration      737 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                   43
Win Rate [%]                            39.53
Best Trade [%]                          18.00
Worst Trade [%]                         -9.77
Avg. Trade [%]                    

In [10]:
stats._strategy

<Strategy SmaCross(n1=5,n2=20)>

In [36]:
bt_sma.plot()

Row(id='5601', ...)

In [46]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=False)
stats = bt_momentum.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

  0%|          | 0/9 [00:00<?, ?it/s]

CPU times: user 582 ms, sys: 18.6 ms, total: 600 ms
Wall time: 588 ms


Start                     2023-07-10 00:00...
End                       2024-07-17 00:00...
Duration                    373 days 00:00:00
Exposure Time [%]                   16.042781
Equity Final [$]                  1462.974438
Equity Peak [$]                       10000.0
Return [%]                         -85.370256
Buy & Hold Return [%]               80.531833
Return (Ann.) [%]                  -84.677677
Volatility (Ann.) [%]                3.859311
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -85.370256
Avg. Drawdown [%]                  -85.370256
Max. Drawdown Duration      368 days 00:00:00
Avg. Drawdown Duration      368 days 00:00:00
# Trades                                   58
Win Rate [%]                         3.448276
Best Trade [%]                       2.959956
Worst Trade [%]                      -9.04646
Avg. Trade [%]                    

In [47]:
bt_momentum.plot()

Row(id='7361', ...)

In [49]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=True)
stats = bt_momentum.run()
stats

Start                     2023-07-10 00:00...
End                       2024-07-17 00:00...
Duration                    373 days 00:00:00
Exposure Time [%]                   16.042781
Equity Final [$]                  1466.796162
Equity Peak [$]                       10000.0
Return [%]                         -85.332038
Buy & Hold Return [%]               80.531833
Return (Ann.) [%]                  -84.638615
Volatility (Ann.) [%]                3.854725
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -85.332038
Avg. Drawdown [%]                  -85.332038
Max. Drawdown Duration      369 days 00:00:00
Avg. Drawdown Duration      369 days 00:00:00
# Trades                                   58
Win Rate [%]                         3.448276
Best Trade [%]                       2.797872
Worst Trade [%]                     -9.037462
Avg. Trade [%]                    